# Fluxo ETL de dados da camada Bronze para camada Silver

## Introdução

Na camada Bronze, os dados brutos, ou seja, sem tratamento e limpeza de dados estão armazenados no arquivo `TMDB_movie_dataset_v11.csv`. Nessa pipeline, serão realizadas as seguintes operações:

- **Limpar e padronizar os dados**: tratar valores nulos, duplicados e NaN;
- **Normalizar formatos**: datas, categorias e colunas numéricas;
- **Enriquecer ou derivar novas colunas** quando necessário para análises futuras;
- **Garantir a qualidade dos dados** antes do carregamento na Silver, que terá dados mais estruturados e prontos para consumo analítico.

Na camada Silver ficam armazenados os **dados tratados e consistentes**, que poderão ser utilizados em análises exploratórias e os dashboards no Tableau e PowerBI.  

As etapas do fluxo ETL são:

1. Carregamento dos dados Bronze (CSV bruto);
2. Inspeção e validação inicial das colunas;
3. Remoção de colunas que não serão usadas;
4. Tratamento de valores nulos, vazios, duplicados e NaN;
5. Transformações de dados e enriquecimentos;
6. Armazenamento dos dados tratados na camada Silver em csv;
7. Carregamento no banco de dados.


## Leitura do csv bruto da camada Bronze

### Import das bibliotecas, configurações iniciais e inicialização da sessão spark

In [7]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode_outer, arrays_zip, col, trim, ltrim, lit, count
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, BooleanType, DateType
import time

# Inicio sessão spark
spark = SparkSession.builder.appName("tmdbEtlBronzeToSilver").getOrCreate()

# Setando data e hora atuais para identificação nos arquivos de output caso necessário
time.tzset()
date_formatted = time.strftime("%Y%m%d")
time_formatted = time.strftime("%H%M%S")
timestamp_str = time.strftime("%Y-%m-%d %H:%M:%S")

### Leitura do csv Bruto

Definição do schema do CSV para importação no pyspark dataframe e leitura do arquivo.

In [ ]:
# Setando o schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("vote_average", FloatType(), True),
    StructField("vote_count", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("release_date", DateType(), True),
    StructField("revenue", IntegerType(), True),
    StructField("runtime", IntegerType(), True),
    StructField("adult", BooleanType(), True),
    StructField("backdrop_path", StringType(), True),
    StructField("budget", IntegerType(), True),
    StructField("homepage", StringType(), True),
    StructField("imdb_id", StringType(), True),
    StructField("original_language", StringType(), True),
    StructField("original_title", StringType(), True),
    StructField("overview", StringType(), True),
    StructField("popularity", FloatType(), True),
    StructField("poster_path", StringType(), True),
    StructField("tagline", StringType(), True),
    StructField("genres", StringType(), True),
    StructField("production_companies", StringType(), True),
    StructField("production_countries", StringType(), True),
    StructField("spoken_languages", StringType(), True)
])

# Ingestão do arquivo
csv_path = "./bronze/TMDB_movie_dataset_v11.csv" 
df = spark.read.csv(csv_path, header=True, schema=schema, sep=',', quote='"', escape='"')

TypeError: DataFrameReader.csv() got an unexpected keyword argument 'stringescape'

In [10]:
df.show(truncate=False)

+------+-------------------------------------------------+------------+----------+--------+------------+----------+-------+-----+--------------------------------+---------+-------------------------------------------------------------------+---------+-----------------+-------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------------------------+--------------------------------------------------------------------------------------+-------------------------------------------+----------------

25/10/11 10:06:00 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 24, schema size: 23
CSV file: file:///home/papercut/Documentos/projects/unb/2025-2/SBD2/film-data-analytics/bronze/TMDB_movie_dataset_v11.csv
